In [7]:
LIST_OF_DIRECTORIES = ["output/cafe_bas/rir", "output/cafe_droite/rir", "output/cafe_gauche/rir", "output/cafe_haut/rir",
                       "output/cafe_milieu/rir", "output/pas_cafe_bas/rir", "output/pas_cafe_droite/rir", 
                       "output/pas_cafe_gauche/rir", "output/pas_cafe_haut/rir", "output/pas_cafe_milieu/rir"]
COUNTAIN_START = "output/cafe_"
PIXEL_LENGTH = 90
BAND_WIDTH = 3
NB_SPLIT = 5
EPOCH = 120


# Première étape : charger les données

Importer le nom des données

In [8]:
import os 
import numpy as np
import random
import rasterio
from sklearn.model_selection import StratifiedKFold
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'

In [9]:
images_name = []
for directory in LIST_OF_DIRECTORIES:
    for filename in os.listdir(directory): 
        images_name.append(directory + "/" + filename)
random.shuffle(images_name)

In [10]:
def reshape(image):
    reshaped_image = []
    for i in range(len(image[0])):
        reshaped_row = []
        for j in range(len(image[0][0])):
            reshaped_cell = []
            reshaped_cell.append(image[0][i][j])
            reshaped_cell.append(image[1][i][j])
            reshaped_cell.append(image[2][i][j])
            reshaped_row.append(reshaped_cell)
        reshaped_image.append(reshaped_row)
    return reshaped_image

Importer les images et leur résultat voulu

In [11]:
images_x = []
images_y = []
for image_name in images_name:
    images_x.append(reshape(rasterio.open(image_name).read()))
    if image_name.startswith(COUNTAIN_START):
        images_y.append(1)
    else:
        images_y.append(0)
images_x = np.array(images_x)
images_y = np.array(images_y)

NameError: name 'image_x' is not defined

In [12]:
print(len(images_x))

1115


# Deuxième étape : créer notre cross validation

In [6]:
from sklearn.model_selection import StratifiedKFold

In [7]:
kfold = StratifiedKFold(n_splits=NB_SPLIT)
print(images_x.shape)

1115
90
90
3
[3586, 2160, 1274]
[[[3586 3298 3682 ... 3654 3506 3550]
  [3392 3152 3552 ... 3594 3492 3576]
  [3204 3264 3594 ... 3410 3530 3626]
  ...
  [ 403  411  376 ...  593  595  529]
  [ 472  467  408 ...  532  526  485]
  [ 490  468  495 ...  491  493  494]]

 [[2160 1678 2136 ... 2502 2322 2254]
  [2080 1788 2318 ... 2626 2228 2382]
  [1766 1980 2254 ... 2302 2424 2546]
  ...
  [ 490  547  539 ...  805  783  787]
  [ 595  623  567 ...  723  720  710]
  [ 684  646  653 ...  666  660  643]]

 [[1274 1062 1396 ... 1364 1256 1318]
  [1200 1086 1290 ... 1420 1262 1326]
  [1062 1228 1416 ... 1304 1348 1350]
  ...
  [ 901  952  909 ... 1194 1178 1138]
  [1013 1009  943 ... 1108 1114 1052]
  [1038 1012 1004 ... 1022 1054 1060]]]


# Troisième étape : entrainer et tester

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
cvscores = []
i = 0
for train, test in kfold.split(images_x, images_y):
    # create model
    model = Sequential()
    model.add(Conv2D(256, (3,3), activation='relu', input_shape=(PIXEL_LENGTH,PIXEL_LENGTH, BAND_WIDTH)))
    model.add(Conv2D(128, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    model.fit(images_x[train], images_y[train], epochs=EPOCH)
    # evaluate the model
    scores = model.evaluate(images_x[test], images_y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    model.save("model"+str(i)+".h5")
    i += 1
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Epoch 1/120
892/892 [==============================] - 125s 140ms/step - loss: 432.8285 - accuracy: 0.5975
Epoch 2/120
892/892 [==============================] - 125s 140ms/step - loss: 0.6642 - accuracy: 0.6312
Epoch 3/120
892/892 [==============================] - 125s 141ms/step - loss: 0.6519 - accuracy: 0.6110
Epoch 4/120
892/892 [==============================] - 125s 140ms/step - loss: 0.6491 - accuracy: 0.6188
Epoch 5/120
892/892 [==============================] - 124s 140ms/step - loss: 0.6457 - accuracy: 0.6177
Epoch 6/120
892/892 [==============================] - 123s 137ms/step - loss: 0.6208 - accuracy: 0.6300
Epoch 7/120
892/892 [==============================] - 122s 137ms/step - loss: 0.6190 - accuracy: 0.6222
Epoch 8/120
892/892 [==============================] - 122s 137ms/step - loss: 0.6086 - accuracy: 0.6413
Epoch 9/120
892/892 [==============================] - 122s 137ms/step - loss: 0.6154 - accuracy: 0.6345
Epoch 10/120
892/892 [==============================]

In [ ]:
#from keras.models import load_model
# load model
#model = load_model('model0.h5')
# summarize model.
#model.summary()